In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ctai-ctd-hackathon/train.csv
/kaggle/input/ctai-ctd-hackathon/test.csv


In [2]:
import os
os.environ["CUML_LOG_LEVEL"] = "error"

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [3]:
%load_ext cuml.accel
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import (
    RandomForestClassifier,
    ExtraTreesClassifier,
    GradientBoostingClassifier,
    HistGradientBoostingClassifier,
    VotingClassifier,
    StackingClassifier
)
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    mean_absolute_error, mean_squared_error, r2_score
)

[2025-08-26 16:38:03.437] [CUML] [info] cuML: Installed accelerator for sklearn.


2025-08-26 16:38:18.571225: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756226298.805084      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756226298.869489      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[2025-08-26 16:38:39.729] [CUML] [info] cuML: Installed accelerator for umap.
[2025-08-26 16:38:39.822] [CUML] [info] cuML: Installed accelerator for hdbscan.
[2025-08-26 16:38:39.822] [CUML] [info] cuML: Successfully initialized accelerator.


In [4]:
df = pd.read_csv("/kaggle/input/ctai-ctd-hackathon/train.csv")
df.drop(columns=["id","PROJECTNUMBER","invoiceId","invoiceDate","ItemDescription","CONSTRUCTION_START_DATE","SUBSTANTIAL_COMPLETION_DATE"])

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,MW,PROJECT_CITY,STATE,PROJECT_COUNTRY,CORE_MARKET,PROJECT_TYPE,SIZE_BUILDINGSIZE,NUMFLOORS,NUMROOMS,NUMBEDS,invoiceTotal,MasterItemNo,QtyShipped,UOM,ExtendedQuantity,PriceUOM,UnitPrice,ExtendedPrice,REVISED_ESTIMATE
0,NaN,Mumbai,Maharashtra,India,Enterprise,Workspace,9000.0,0.0,0.0,0.0,1.599409e+05,58211,2,can,2,NaN,1983.125,NaN,2.409524e+07
1,NaN,Delhi,Delhi,India,Wellness,Health Center,31200.0,NaN,NaN,NaN,5.894733e+05,53691,15,PC,0.15,ML,53273.000,7.990950e+03,9.511661e+08
2,NaN,Bengaluru,Karnataka,India,Enterprise,Workspace,2800000.0,0.0,0.0,0.0,5.444225e+05,63311,5,BG,5,BG,864.225,4.321125e+03,3.492557e+10
3,NaN,Chennai,Tamil Nadu,India,Tertiary Learning,Learning Hub,184000.0,0.0,0.0,0.0,2.247720e+05,61506,12,EA,12,EA,125.250,1.503000e+03,1.553978e+10
4,NaN,Hyderabad,Telangana,India,Wellness,Ambulatory Care,25000.0,NaN,NaN,NaN,5.926412e+04,50921,8,PC,0.128,NaN,NaN,2.378080e+04,6.867939e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14031,10.0,Kolkata,West Bengal,India,Future Tech,Data Center,174000.0,6.0,NaN,NaN,4.162891e+08,PP_CAT6_48,460,EA,460,EA,10113.520,4.652219e+06,8.988177e+09
14032,10.0,Kolkata,West Bengal,India,Future Tech,Data Center,174000.0,6.0,NaN,NaN,2.949775e+08,PANELBOARD_LV,184,EA,184,EA,210844.180,3.879533e+07,8.988177e+09
14033,10.0,Kolkata,West Bengal,India,Future Tech,Data Center,174000.0,6.0,NaN,NaN,2.949775e+08,RAISED_FLOOR_SF,18622,SF,18622,SF,7491.620,1.395089e+08,8.988177e+09
14034,10.0,Kolkata,West Bengal,India,Future Tech,Data Center,174000.0,6.0,NaN,NaN,2.949775e+08,CHILLER_TON,2632,TON,2632,TON,42203.405,1.110794e+08,8.988177e+09


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14036 entries, 0 to 14035
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           14036 non-null  int64  
 1   PROJECTNUMBER                14036 non-null  object 
 2   MW                           3300 non-null   float64
 3   PROJECT_CITY                 14036 non-null  object 
 4   STATE                        14036 non-null  object 
 5   PROJECT_COUNTRY              14036 non-null  object 
 6   CORE_MARKET                  14036 non-null  object 
 7   PROJECT_TYPE                 14034 non-null  object 
 8   SIZE_BUILDINGSIZE            14006 non-null  float64
 9   NUMFLOORS                    9587 non-null   float64
 10  NUMROOMS                     6744 non-null   float64
 11  NUMBEDS                      5909 non-null   float64
 12  invoiceId                    14036 non-null  object 
 13  invoiceDate     

In [6]:
X = df.drop(columns=["MasterItemNo","QtyShipped"])
le = LabelEncoder()
cat = le.fit_transform(df["MasterItemNo"])
reg = pd.to_numeric(df["QtyShipped"], errors="coerce")
reg = reg.fillna(reg.median())

In [7]:
cat_features = X.select_dtypes(include=["object"]).columns.tolist()
num_features = X.select_dtypes(exclude=["object"]).columns.tolist()

In [8]:
# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False, dtype=np.float32), cat_features),
        ("num", Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler())
        ]), num_features),
    ]
)

In [9]:
# For classification task
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(
    X, cat, test_size=0.2, random_state=42,
)

# For regression task
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X, reg, test_size=0.2, random_state=42
)

In [10]:
# models = {
#     # "LogisticRegression": LogisticRegression(max_iter=1000, verbose=0),  # CPU only
#     # "RidgeClassifier": RidgeClassifier(),  # CPU only
#     ## "RandomForest": RandomForestClassifier(n_estimators=300,
#     #                                        random_state=42, verbose=0),  # CPU, single-thread
#     # "ExtraTrees": ExtraTreesClassifier(n_estimators=300, max_depth=12,
#                                        # random_state=42, verbose=0),  # CPU, single-thread
#     "GradientBoosting": GradientBoostingClassifier(verbose=0),  # CPU only
#     "HistGradientBoosting": HistGradientBoostingClassifier(verbose=0),  # CPU only
#     "SVC": SVC(probability=True, verbose=0),  # CPU (GPU only with cuML, sklearn = CPU)
#     "LinearSVC": LinearSVC(verbose=0),  # CPU only
#     "KNN": KNeighborsClassifier(n_neighbors=5),  # CPU, single-thread
#     "GaussianNB": GaussianNB(),  # CPU only
#     "XGBoost": XGBClassifier(tree_method="gpu_hist", predictor="gpu_predictor",
#                              use_label_encoder=False, eval_metric="mlogloss", verbosity=0),  # GPU
#     "LightGBM": LGBMClassifier(device="gpu", verbose=-1),  # GPU
#     "CatBoost": CatBoostClassifier(task_type="GPU", devices="0", verbose=0)  # GPU
# }

In [11]:
# for name, model in models.items():
#     pipe = Pipeline([
#         ("preprocessor", preprocessor),
#         ("clf", model)
#     ])

#     pipe.fit(X_train_cat, y_train_cat)
#     y_pred = pipe.predict(X_test_cat)

#     acc = accuracy_score(y_test_cat, y_pred)

#     print(f"+++++++{name}: Accuracy={acc}=======")

+++++++LogisticRegression: Accuracy=0.41061253561253563=======
+++++++RidgeClassifier: Accuracy=0.8144586894586895=======
+++++++ExtraTrees: Accuracy=0.49002849002849=======

In [12]:
model=RidgeClassifier()

In [13]:
pipe_cat = Pipeline([
        ("preprocessor", preprocessor),
        ("clf", model)
    ])

In [14]:
pipe_cat.fit(X_train_cat, y_train_cat)
#y_pred_cat = pipe.predict(X_test_cat)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(dtype=<class 'numpy.float32'>,
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['PROJECTNUMBER',
                                                   'PROJECT_CITY', 'STATE',
                                                   'PROJECT_COUNTRY',
                                                   'CORE_MARKET',
                                                   'PROJECT_TYPE', 'invoiceId',
                                                   'invoiceDate',
                                                   'ItemDescription', 'UOM',
                                                   'ExtendedQuantity',
                                                   'PriceUOM',
                                                   'CONSTRUCTION_START_DATE',
                                                   'SUBSTANTIAL_COMPLETION_DATE']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['id', 'MW',
                                                   'SIZE_BUILDINGSIZE',
                                                   'NUMFLOORS', 'NUMROOMS',
                                                   'NUMBEDS', 'invoiceTotal',
                                                   'UnitPrice', 'ExtendedPrice',
                                                   'REVISED_ESTIMATE'])])),
                ('clf', RidgeClassifier())])

In [15]:
# from sklearn.base import BaseEstimator, TransformerMixin
# import numpy as np
# import pandas as pd

# class NaNChecker(BaseEstimator, TransformerMixin):
#     def __init__(self, feature_names=None):
#         self.feature_names = feature_names  # Pass feature names if available

#     def fit(self, X, y=None):
#         return self
    
#     def transform(self, X):
#         # Convert sparse -> dense if needed
#         if hasattr(X, "toarray"):
#             X = X.toarray()

#         # If feature names are known
#         if self.feature_names is not None:
#             df = pd.DataFrame(X, columns=self.feature_names)
#             nan_cols = df.columns[df.isna().any()].tolist()
#             if nan_cols:
#                 print(f"⚠️ NaNs detected in columns: {nan_cols}")
#             else:
#                 print("✅ No NaNs found after preprocessing")
#         else:
#             # Just report index positions
#             nan_indices = np.where(np.isnan(X))[1]
#             if nan_indices.size > 0:
#                 print(f"⚠️ NaNs detected at feature indices: {np.unique(nan_indices)}")
#             else:
#                 print("✅ No NaNs found after preprocessing")
        
#         return X

In [16]:
# from cuml.linear_model import LinearRegression, Ridge, Lasso
# from cuml.ensemble import RandomForestRegressor
# import xgboost as xgb
# import lightgbm as lgb
# from catboost import CatBoostRegressor
# from cuml.svm import SVR
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# regression_models = {
#     "LinearRegression": LinearRegression(),
#     "Ridge": Ridge(alpha=1.0),
#     "Lasso": Lasso(alpha=0.1),
#     "RandomForest": RandomForestRegressor(n_estimators=200, max_depth=16),
#     "XGBoost": xgb.XGBRegressor(tree_method="gpu_hist", n_estimators=500, max_depth=8),
#     "LightGBM": lgb.LGBMRegressor(device="gpu", n_estimators=500, max_depth=-1),
#     "CatBoost": CatBoostRegressor(iterations=500, depth=8, task_type="GPU", verbose=0),
#     "SVR": SVR(C=1.0, kernel="rbf"),
# }

In [17]:
# results = []

# for name, model in regression_models.items():
#     try:
#         # Fit preprocessor first to get feature names
#         preprocessor.fit(X_train_reg)
#         feature_names = preprocessor.get_feature_names_out()

#         # Build pipeline with preprocessing + NaNChecker + model
#         pipe = Pipeline([
#             ("preprocessor", preprocessor),
#             ("regressor", model)
#         ])

#         # Fit the full pipeline
#         pipe.fit(X_train_reg, y_train_reg)

#         # Predict
#         y_pred = pipe.predict(X_test_reg)

#         # Convert GPU output to numpy if needed
#         if hasattr(y_pred, "get"):
#             y_pred = y_pred.get()

#         # Metrics
#         r2 = r2_score(y_test_reg, y_pred)
#         rmse = mean_squared_error(y_test_reg, y_pred, squared=False)
#         mae = mean_absolute_error(y_test_reg, y_pred)

#         results.append([name, r2, rmse, mae])
#         print(f"+++++ {name:<15} | R²={r2:.4f}, RMSE={rmse:.4f}, MAE={mae:.4f}")

#     except Exception as e:
#         print(f"⚠️ {name} failed: {e}")

# # Collect results in a table
# df_results = pd.DataFrame(results, columns=["Model", "R²", "RMSE", "MAE"])
# print("\n===== Regression Summary =====")
# print(df_results)

+++++ LinearRegression | R²=0.2538, RMSE=825.3891, MAE=179.3362
+++++ Ridge           | R²=0.4994, RMSE=676.0743, MAE=228.3340
+++++ Lasso           | R²=0.2525, RMSE=826.1263, MAE=157.4389
+++++ RandomForest    | R²=0.3415, RMSE=775.3722, MAE=242.8549
+++++ XGBoost         | R²=0.9670, RMSE=173.4498, MAE=25.3283
+++++ LightGBM        | R²=0.9507, RMSE=212.1010, MAE=45.5969
+++++ CatBoost        | R²=0.9747, RMSE=152.0886, MAE=53.0906
+++++ SVR             | R²=-0.0062, RMSE=958.4589, MAE=195.3034

In [18]:
from catboost import CatBoostRegressor
model2 = CatBoostRegressor(iterations=500, depth=8, task_type="GPU", verbose=0)

pipe_reg = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", model2)
])

pipe_reg.fit(X_train_reg, y_train_reg)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(dtype=<class 'numpy.float32'>,
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['PROJECTNUMBER',
                                                   'PROJECT_CITY', 'STATE',
                                                   'PROJECT_COUNTRY',
                                                   'CORE_MARKET',
                                                   'PROJECT_TYPE', 'invoiceId',
                                                   'invoiceDate',
                                                   'ItemDescription', 'UOM',
                                                   'ExtendedQuantity',
                                                   'PriceUOM',
                                                   'CONSTRUCTION_START_DATE',
                                                   'SUBSTANTIAL_COMPLETION_DATE']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['id', 'MW',
                                                   'SIZE_BUILDINGSIZE',
                                                   'NUMFLOORS', 'NUMROOMS',
                                                   'NUMBEDS', 'invoiceTotal',
                                                   'UnitPrice', 'ExtendedPrice',
                                                   'REVISED_ESTIMATE'])])),
                ('regressor',
                 <catboost.core.CatBoostRegressor object at 0x7ef90d6b7c90>)])

+++++++++++++++++++++++++++++++++TEST+++++++++++++++++++++++++++

In [19]:
df = pd.read_csv("/kaggle/input/ctai-ctd-hackathon/test.csv")
df.drop(columns=["id","PROJECTNUMBER","invoiceId","invoiceDate","ItemDescription","CONSTRUCTION_START_DATE","SUBSTANTIAL_COMPLETION_DATE"])

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,MW,PROJECT_CITY,STATE,PROJECT_COUNTRY,CORE_MARKET,PROJECT_TYPE,SIZE_BUILDINGSIZE,NUMFLOORS,NUMROOMS,NUMBEDS,invoiceTotal,UOM,ExtendedQuantity,PriceUOM,UnitPrice,ExtendedPrice,REVISED_ESTIMATE
0,NaN,Visakhapatnam,Andhra Pradesh,India,Enterprise,Critical Ops,265850.0,0.0,0.0,0.0,142835.935,roll,2,NaN,2004.000,NaN,4.131524e+09
1,NaN,Lucknow,Uttar Pradesh,India,Bio Innovation,R&amp;D Laboratories,288000.0,0.0,0.0,0.0,1393673.450,PC,1.275,ML,140280.000,178857.835,1.559882e+10
2,NaN,Indore,Madhya Pradesh,India,Misc Market,Learning Hub,40000.0,2.0,0.0,0.0,1446021.270,EA,36,EA,165.330,5951.880,3.600137e+09
3,NaN,Warangal,Telangana,India,Misc Market,Learning Hub,36700.0,4.0,0.0,0.0,10460.880,NaN,NaN,NaN,4178.340,8356.680,3.422320e+09
4,NaN,Kolkata,West Bengal,India,Enterprise,Hospitality Hall,435000.0,11.0,500.0,NaN,192062.525,PC,0.22,ML,157230.500,34590.710,2.320456e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2680,NaN,NaN,West Bengal,India,Wellness,Ambulatory Care,17000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,172550.245,NaN,1.591057e+09
2681,NaN,Patna,Bihar,India,Wellness,Health Center,12658.0,0.0,0.0,0.0,180418.450,ctn,2240,NaN,75985.000,NaN,2.656048e+08
2682,NaN,Kolkata,West Bengal,India,Enterprise,Hospitality Hall,435000.0,11.0,500.0,NaN,185708.175,PC,0.06,ML,180777.500,10846.650,2.320456e+10
2683,NaN,Coimbatore,Tamil Nadu,India,Tertiary Learning,Misc Build,34000.0,2.0,NaN,NaN,538485.655,PC,0.5,ML,194722.000,97361.000,1.642866e+09


In [20]:
X = df

In [21]:
myregpred=pipe_reg.predict(X)

In [22]:
mycatpred_indices = pipe_cat.predict(X)
mycatpred_labels = le.inverse_transform(mycatpred_indices)

# Regression predictions are already on X
myregpred = pipe_reg.predict(X)

# Create final submission
final_submission = pd.DataFrame({
    'id': df['id'],  # make sure df still has 'id' column
    'MasterItemNo': mycatpred_labels,
    'QtyShipped': myregpred
})

final_submission.to_csv("submission.csv", index=False)